In [2]:
import numpy as np
from xgboost import XGBRegressor
from sklearn.datasets import load_svmlight_file
from collections import defaultdict
from multiprocessing import Pool, Array
import time
import dill
import tqdm
from tqdm import tqdm_notebook as tqdm

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!ln -s gdrive/My\ Drive ./data
path = 'data/data_relevance'
!nvidia-smi

Fri Oct 30 08:50:56 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
def idcg(rel):
    dcg = np.sum((2.0**(np.sort(rel)[::-1]) - 1.0) / (np.log(1.0 + np.arange(rel.shape[0])) + 1.0))
    if np.isclose(dcg, 0.0):
        return 1.0
    else:
        return 1.0 / dcg
def dcg(rel):
    return np.sum((2**rel - 1) / (np.log(1.0 + np.arange(rel.shape[0])) + 1.0))

In [6]:
X, y, query_ids = load_svmlight_file(path + '/train.txt', query_id=True)
X_test, y_test, query_ids_test = load_svmlight_file(path + '/test.txt', query_id=True)

In [7]:
train_queries = dict()

for doc_id, query_id in enumerate(query_ids):
    if query_id in train_queries:
      train_queries[query_id].append(doc_id)
    else:
      train_queries[query_id] = [doc_id]

In [8]:
def queries_bagging(new_ids):
  ids = dict()
  for k, i in enumerate(new_ids):
    if i in ids:
      ids[i].append(k)
    else:
      ids[i] = [k]
  bs_queries = defaultdict(list)
  bs_local = defaultdict(list)
  bs_global = defaultdict(list)
  for query_id in train_queries:
    counter=0
    for d in train_queries[query_id]:
      if d in ids:
        bs_queries[query_id].append(ids[d][0])
        bs_global[query_id].extend(ids[d])
        bs_local[query_id].extend([counter for i in range(len(ids[d]))])
        counter += 1
  return bs_queries, bs_local, bs_global

In [9]:
def utilities(y, bs_queries):
  query_P_ij = dict()
  logs_q = dict()
  idcgs = dict()
  a_powered = dict()
  for query_id in bs_queries:
    docs = np.array(bs_queries[query_id])
    a = y[docs]
    S_ij = np.zeros((a.shape[0], a.shape[0]))
    S_ij += a.reshape(-1, 1) > a
    S_ij -= a.reshape(-1, 1) < a
    P_ij = 0.5*(1 + S_ij)
    logs_ = 1.0/ (np.log(np.arange(a.shape[0]) + 1.0) + 1.0)
    logs_ = logs_.reshape(-1, 1) - logs_
    query_P_ij[query_id] = P_ij
    logs_q[query_id] = logs_
    idcgs[query_id] = idcg(a)
    a_powered[query_id] = 2**a
  return query_P_ij, logs_q, idcgs, a_powered

In [10]:
def get_absent(new_ids):
  s = set(new_ids)
  t = range(0, len(new_ids))
  absent_ids = list(filter(lambda x: x not in s, t))
  return absent_ids

In [11]:
def objective(y_true, y_pred):

    global progress_bar
    progress_bar.update(1)
    global_grad = np.zeros(y_true.shape[0])
    global_hess = np.zeros(y_true.shape[0])
    
    
    
    for query_id in bs_queries:
        docs = np.array(bs_queries[query_id])
        a = y_true[docs]
        h = y_pred[docs]
        
        argsorted = np.argsort(h)
        ts = a_powered[query_id][argsorted]
        tpq = ts.reshape(-1, 1) - ts
        logs = logs_q[query_id]
        deltas = tpq*logs
        idcg_cur = idcgs[query_id]
        deltas = np.abs(idcg_cur*deltas)
        P_ij = query_P_ij[query_id]
        h_ij = h.reshape(-1, 1) - h
        p_ij = -1/(1 + np.exp(h_ij))
        fpq = P_ij*deltas
        T = fpq*p_ij
        grad = np.sum(-T + np.transpose(T), axis=0)
        p_ij_ = np.exp(-h_ij)/ (1 + np.exp(-h_ij))**2
        T_ = fpq*p_ij_
        hess = np.sum(T_ + np.transpose(T_), axis=1) - 2*T_.diagonal()
        global_grad[bs_global[query_id]] = grad[bs_local[query_id]]
        global_hess[bs_global[query_id]] = hess[bs_local[query_id]]
    
    return global_grad, global_hess

In [12]:
def calc_full_ndcg(y_true, y_pred):
  ndcgs = []
  for query_id in train_queries:
    docs = train_queries[query_id]
    ndcg = idcg(y_true[docs])*dcg(y_true[np.argsort(y_pred[docs])])
    ndcgs.append(ndcg)
  return np.mean(ndcgs)

***44 XGBRegressor, n_estimators = 200, max_depth = 16, lr=0.1, public_score = 0.75253***

In [ ]:
y_validation = np.zeros_like(y)
for i in range(0, 44):
  new_ids = np.random.randint(low=0, high = X.shape[0], size=X.shape[0])
  bs_queries, bs_local, bs_global = queries_bagging(new_ids)
  query_P_ij, logs_q, idcgs, a_powered = utilities(y[new_ids], bs_queries)
  params = {'objective': objective, 'max_depth': 16, 'n_estimators': 200, 'tree_method':'gpu_hist', 'lr':0.1}
  model = XGBRegressor(**params)
  progress_bar = tqdm(np.arange(200))
  model.fit(X[new_ids], y[new_ids])
  absent_ids = get_absent(new_ids)
  val_pred = model.predict(X[absent_ids])
  y_validation[absent_ids] += val_pred
  cur_pred = model.predict(X_test)
  dill.dump(y_validation, open(path + '/bagging_dump_200/out_of_bag_validation/' + 'y_validation' + str(i) + '.dump', 'wb'))
  dill.dump(model, open(path + '/bagging_dump_200/model_dumps/' + 'model' + str(i) + '.dump', 'wb'))
  dill.dump(cur_pred, open(path + '/bagging_dump_200/prediction_dumps/' + 'test_prediction' + str(i) + '.dump', 'wb'))
  progress_bar.close()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


[05:55:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.



[06:08:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.



[06:20:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.



[06:33:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.



...

[07:08:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.



[07:20:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.



[07:32:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.



[07:43:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.



[07:56:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [22]:
y_pred = np.zeros_like(y_test)
for i in range(0, 44):
  cur_pred = dill.load(open(path + '/bagging_dump_200/prediction_dumps/test_prediction' + str(i) + '.dump', 'rb'))
  y_pred += cur_pred

***84 XGBRegressor, max_depth=12, n_estimators=300, lr=0.1, public_score=0.75201***

In [24]:
y_validation = np.zeros_like(y)
for i in range(0, 84):
  new_ids = np.random.randint(low=0, high = X.shape[0], size=X.shape[0])
  bs_queries, bs_local, bs_global = queries_bagging(new_ids)
  query_P_ij, logs_q, idcgs, a_powered = utilities(y[new_ids], bs_queries)
  params = {'objective': objective, 'max_depth': 12, 'n_estimators': 300, 'tree_method':'gpu_hist', 'lr':0.1}
  model = XGBRegressor(**params)
  progress_bar = tqdm(np.arange(300))
  model.fit(X[new_ids], y[new_ids])
  absent_ids = get_absent(new_ids)
  val_pred = model.predict(X[absent_ids])
  y_validation[absent_ids] += val_pred
  cur_pred = model.predict(X_test)
  dill.dump(y_validation, open(path + '/bagging_dump_300/out_of_bag_validation/' + 'y_validation' + str(i) + '.dump', 'wb'))
  dill.dump(model, open(path + '/bagging_dump_300/model_dumps/' + 'model' + str(i) + '.dump', 'wb'))
  dill.dump(cur_pred, open(path + '/bagging_dump_300/prediction_dumps/' + 'test_prediction' + str(i) + '.dump', 'wb'))
  progress_bar.close()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


[09:38:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.



[09:52:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.



In [25]:
y_pred = np.zeros_like(y_test)
for i in range(0, 84):
  cur_pred = dill.load(open(path + '/bagging_dump_300/prediction_dumps/test_prediction' + str(i) + '.dump', 'rb'))
  y_pred += cur_pred

_________________________________________________________________________________________________

In [26]:
test_queries = defaultdict(list)
for doc_id, query_id in enumerate(query_ids_test):
  test_queries[query_id].append(doc_id)

In [28]:
with open(path + '/submission_f.csv', 'w') as submission:
  string = "QueryId,DocumentId\n"
  for query_id in test_queries:
    docs = test_queries[query_id]
    y_pred_cur = y_pred[docs]
    sorted_ids = np.argsort(y_pred_cur)[::-1]
    sorted_docs = np.array(docs)[sorted_ids]
    for doc_id in sorted_docs:
      string += str(query_id) + ',' + str(doc_id + 1) + '\n'
  submission.write(string)